In [ ]:
# Downloading the nature_12K image dataset from the this URL and save it as 'nature_12K.zip'
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip
#Remove the zip file after extraction to free up disk space
!rm nature_12K.zip

--2025-04-19 13:54:46--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   225MB/s    in 24s     

2025-04-19 13:55:10 (150 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import os

# ========== Setup ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

num_classes = 10
batch_size = 32
num_epochs = 5
data_dir = 'inaturalist_12K'

# ========== Transforms ==========
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



Using device: cuda


In [ ]:
# ========== Dataset and Dataloaders ==========
full_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# ========== Fine-Tuning Function ==========
def setup_finetune_model(strategy='unfreeze_last2'):
    model = models.resnet50(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    if strategy == 'freeze_all':
        for param in model.parameters():
            param.requires_grad = False
        for param in model.fc.parameters():
            param.requires_grad = True

    elif strategy == 'unfreeze_last2':
        for name, param in model.named_parameters():
            if 'layer3' in name or 'layer4' in name or 'fc' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

    elif strategy == 'unfreeze_all':
        for param in model.parameters():
            param.requires_grad = True

    return model.to(device)


In [ ]:
# ========== Training and Evaluation ==========
def train(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(loader, desc='Training'):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Evaluating'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

In [ ]:
# ========== Main Training Loop ==========
strategy = 'unfreeze_last2'  # can be: freeze_all, unfreeze_last2, unfreeze_all
model = setup_finetune_model(strategy)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

print(f"\n== Fine-tuning ResNet50 with strategy: {strategy} ==")
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    val_loss, val_acc = evaluate(model, val_loader, criterion)
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# ========== Final Test Accuracy ==========
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"\n== Final Test Results ==")
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 165MB/s]



== Fine-tuning ResNet50 with strategy: unfreeze_last2 ==

Epoch 1/5


Training: 100%|██████████| 250/250 [01:31<00:00,  2.73it/s]


Train Loss: 0.9511, Train Acc: 0.6952


Evaluating: 100%|██████████| 63/63 [00:20<00:00,  3.07it/s]


...

Train Loss: 0.0874, Train Acc: 0.9756


Evaluating: 100%|██████████| 63/63 [00:20<00:00,  3.06it/s]


Val Loss: 0.9988, Val Acc: 0.7440


Evaluating: 100%|██████████| 63/63 [00:20<00:00,  3.07it/s]


== Final Test Results ==
Test Loss: 0.9278, Test Acc: 0.7590


In [ ]:
strategies = ['freeze_all', 'unfreeze_last2', 'unfreeze_all']
results = {}
for strategy in strategies:
    print(f"\n=== Running strategy: {strategy} ===")
    model = setup_finetune_model(strategy)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        train_loss, train_acc = train(model, train_loader, optimizer, criterion)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

        val_loss, val_acc = evaluate(model, val_loader, criterion)
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Final Test Accuracy
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"\n>>> [Strategy: {strategy}] Test Accuracy: {test_acc:.4f}")
    results[strategy] = test_acc

# Print summary
print("\n===== Final Accuracy Comparison =====")
for strategy, acc in results.items():
    print(f"{strategy:>15}: {acc:.4f}")


=== Running strategy: freeze_all ===

Epoch 1/5


Training: 100%|██████████| 250/250 [01:20<00:00,  3.09it/s]


Train Loss: 1.7954, Train Acc: 0.4989


Evaluating: 100%|██████████| 63/63 [00:20<00:00,  3.07it/s]


Val Loss: 1.3818, Val Acc: 0.6650

Epoch 2/5


...

Train Loss: 0.1413, Train Acc: 0.9545


Evaluating: 100%|██████████| 63/63 [00:20<00:00,  3.13it/s]


Val Loss: 0.9232, Val Acc: 0.7520


Evaluating: 100%|██████████| 63/63 [00:20<00:00,  3.07it/s]


>>> [Strategy: unfreeze_all] Test Accuracy: 0.7595

===== Final Accuracy Comparison =====
     freeze_all: 0.7375
 unfreeze_last2: 0.7735
   unfreeze_all: 0.7595
